Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff, LinearAlgebra

### snippet 10.4

In [2]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
first(df, 5)

struct m_10_02d_model{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_10_02d_model)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack pr, β = θ            # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(0, 10), β[1]) # a = X[1]
    ll += logpdf(Normal(0, 10), β[2]) # bp = X[2]
    ll += sum([loglikelihood(Binomial(1, logistic(dot(X[i, :], β))), [y[i]]) for i in 1:N])
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
X = hcat(ones(Int64, N), df[:prosoc_left]);
y = df[:pulled_left]
p = m_10_02d_model(y, X);
θ = (β = [1.0, 2.0], pr = ones(N),)
p(θ)

-487.6540051035728

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::m_10_02d_model) =
    as((β = as(Array, size(p.X, 2)), pr = as(Array, size(p.X, 2))))
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.08 s/step ...done
MCMC, adapting ϵ (25 steps)
0.083 s/step ...done
MCMC, adapting ϵ (50 steps)
0.083 s/step ...done
MCMC, adapting ϵ (100 steps)
step 100 (of 100), 0.075 s/step
0.075 s/step ...done
MCMC, adapting ϵ (200 steps)
step 100 (of 200), 0.077 s/step
step 200 (of 200), 0.077 s/step
0.077 s/step ...done
MCMC, adapting ϵ (400 steps)
step 100 (of 400), 0.081 s/step
step 200 (of 400), 0.083 s/step
step 300 (of 400), 0.084 s/step
step 400 (of 400), 0.086 s/step
0.086 s/step ...done
MCMC, adapting ϵ (50 steps)
0.08 s/step ...done
MCMC (1000 steps)
step 100 (of 1000), 0.099 s/step
step 200 (of 1000), 0.097 s/step
step 300 (of 1000), 0.095 s/step
step 400 (of 1000), 0.094 s/step
step 500 (of 1000), 0.094 s/step
step 600 (of 1000), 0.093 s/step
step 700 (of 1000), 0.094 s/step
step 800 (of 1000), 0.093 s/step
step 900 (of 1000), 0.094 s/step
step 1000 (of 1000), 0.093 s/step
0.093 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :pr),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [-0.0525353, 0.703669], pr = [-28685.5, -6714.82])
 (β = [-0.0145599, 0.690931], pr = [-28665.9, -6739.36])
 (β = [-0.0881848, 0.735128], pr = [-29124.6, -6543.31])
 (β = [0.0716152, 0.466793], pr = [-29827.7, -7658.12]) 
 (β = [-0.0161344, 0.592339], pr = [-29240.3, -5136.78])

Extract the parameter posterior means: `β`,

In [8]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.05027976744800832
 0.555398298499633  

Effective sample sizes (of untransformed draws)

In [9]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)
# NUTS-specific statistics
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.95, min/25%/median/75%/max: 0.55 0.93 0.98 1.0 1.0
  termination: MaxDepth => 76% AdjacentTurn => 15% DoubledTurn => 9%
  depth: 1 => 0% 2 => 9% 3 => 8% 4 => 2% 5 => 1% 6 => 0% 7 => 1% 8 => 1% 9 => 2% 10 => 76%


CmdStan result

In [10]:
m_10_2s_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
      Mean        SD       Naive SE       MCSE      ESS
 a 0.05103234 0.12579086 0.0019889282 0.0035186307 1000
bp 0.55711212 0.18074275 0.0028577937 0.0040160451 1000

Quantiles:
       2.5%        25.0%       50.0%      75.0%      97.5%
 a -0.19755400 -0.029431425 0.05024655 0.12978825 0.30087758
bp  0.20803447  0.433720250 0.55340400 0.67960975 0.91466915
";

Extract the parameter posterior means: `β`,

In [11]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.05027976744800832
 0.555398298499633  

End of `10/m10.02d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*